In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
from sklearn.datasets import load_boston
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [4]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [5]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [6]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [7]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [8]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


In [9]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,0,0,2


In [10]:

X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [11]:
#OneHotEncoding
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [12]:
X_train

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
103291,1274,4190,2,25,8,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
233923,1207,12014,1,10,9,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
166653,944,8232,3,40,7,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
150634,488,12448,2,0,7,5,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
82720,302,5339,1,10,5,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,217,10644,1,25,4,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17730,600,4813,2,20,13,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
28030,463,4692,2,10,9,4,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
15725,600,157,2,50,5,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
import time

# min_child_weight = [0, 1, 2]
# max_delta_step = [0, 5, 10]

def my_grid_search():
    print(time.gmtime())
    i = 1
    df = pd.DataFrame({'subsample': [],
                       'gamma': [],
                       'learning_rate': [],
                       'max_depth': [],
                       'score': []})
    for subsample in [0.75, 0.885, 0.95]:
        for gamma in [0.75, 1, 1.25]:
            for learning_rate in [0.4375, 0.45, 0.4625]:
                 for max_depth in [5, 6, 7]:
                    model = XGBClassifier(n_estimators = 350,
                                          booster = 'gbtree',
                                          subsample = subsample,
                                          gamma = gamma,
                                          max_depth = max_depth,
                                          learning_rate = learning_rate,
                                          label_encoder = False,
                                          verbosity = 0)
                    model.fit(X_train, y_train)
                    y_preds = model.predict(X_test)
                    score = f1_score(y_test, y_preds, average = 'micro')
                    df = df.append(pd.Series(
                        data={'subsample': subsample,
                              'gamma': gamma,
                              'learning_rate': learning_rate,
                              'max_depth': max_depth,
                              'score': score},
                    name = i))
                    print(i, time.gmtime())
                    i += 1

    return df.sort_values('score', ascending = False)

current_df = my_grid_search()
df = pd.read_csv('grid-search/res-feature-engineering.csv')
df.append(current_df)
df.to_csv('grid-search/res-feature-engineering.csv')

current_df

time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=5, tm_min=48, tm_sec=50, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=5, tm_min=50, tm_sec=27, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=5, tm_min=54, tm_sec=28, tm_wday=1, tm_yday=194, tm_isdst=0)


...

/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


143 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=17, tm_min=18, tm_sec=27, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


144 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=17, tm_min=25, tm_sec=54, tm_wday=1, tm_yday=194, tm_isdst=0)


,subsample,gamma,learning_rate,max_depth,score
90,0.885,1.0,0.450,6.0,0.743577
114,0.950,0.5,0.450,6.0,0.743424
18,0.750,1.0,0.450,6.0,0.743232
74,0.885,0.5,0.425,6.0,0.743117
130,0.950,1.0,0.550,6.0,0.743098
...,...,...,...,...,...
97,0.885,1.5,0.425,3.0,0.724199
49,0.825,1.0,0.425,3.0,0.724123
29,0.750,1.5,0.450,3.0,0.724123
12,0.750,0.5,0.550,12.0,0.723969


In [51]:
import time

def my_grid_search():
    print(time.gmtime())
    i = 1
    df = pd.DataFrame({'subsample': [],
                       'gamma': [],
                       'learning_rate': [],
                       'max_depth': [],
                       'score': []})
    for subsample in [0.885]:
        for gamma in [1]:
            for learning_rate in [0.45]:
                for max_depth in [5,6,7,8]:
                    model = XGBClassifier(n_estimators = 350,
                                          booster = 'gbtree',
                                          subsample = subsample,
                                          gamma = gamma,
                                          max_depth = max_depth,
                                          learning_rate = learning_rate,
                                          label_encoder = False,
                                          verbosity = 0)
                    model.fit(X_train, y_train)
                    y_preds = model.predict(X_test)
                    score = f1_score(y_test, y_preds, average = 'micro')
                    df = df.append(pd.Series(
                        data={'subsample': subsample,
                              'gamma': gamma,
                              'learning_rate': learning_rate,
                              'max_depth': max_depth,
                              'score': score},
                    name = i))
                    print(i, time.gmtime())
                    i += 1

    return df.sort_values('score', ascending = False)

df = my_grid_search()
# df = pd.read_csv('grid-search/res-feature-engineering.csv')
# df.append(current_df)
df.to_csv('grid-search/res-feature-engineering.csv')

df

time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=22, tm_sec=10, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=24, tm_sec=23, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=27, tm_sec=28, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


3 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=41, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


4 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=37, tm_sec=7, tm_wday=1, tm_yday=194, tm_isdst=0)


,subsample,gamma,learning_rate,max_depth,score
2,0.885,1.0,0.45,6.0,0.743577
3,0.885,1.0,0.45,7.0,0.742868
4,0.885,1.0,0.45,8.0,0.740968
1,0.885,1.0,0.45,5.0,0.740719


In [14]:
pd.read_csv('grid-search/res-no-feature-engineering.csv')\
    .nlargest(20, 'score')

FileNotFoundError: [Errno 2] No such file or directory: 'grid-search/res-no-feature-engineering.csv'

# ...

In [33]:
sgd_model = BernoulliNB()

In [34]:
sgd_model.fit(X_train, y_train)

BernoulliNB()

In [35]:
X_test.shape

(52121, 98)

In [36]:
y_preds = sgd_model.predict(X_test)
f1_score(y_test, y_preds, average='micro')

0.6242013775637459

In [45]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [46]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")
test_values_subset

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [47]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)
test_values_subset

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,596,11307,3,20,7,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
99355,141,11987,2,25,13,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
890251,19,10044,2,5,4,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
745817,39,633,1,0,19,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,289,7970,3,15,8,7,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,605,3623,3,70,20,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
663567,1407,11907,3,25,6,7,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1049160,1136,7712,1,50,3,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [48]:
test_values_subset.shape

(86868, 98)

In [49]:
# Genero las predicciones para los test.
preds = vc_model.predict(test_values_subset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    3.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.8s finished


In [50]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [51]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [52]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [53]:
my_submission.to_csv('../../csv/predictions/jf/vote/jf-model-3-submission.csv')

In [54]:
!head ../../csv/predictions/jf/vote/jf-model-3-submission.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,1
896100,3
343471,2
